In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [11]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(return_messages=True, k=2)

In [12]:
from langchain_chinese import WithMemoryBinding

withMemoryChain = WithMemoryBinding(
  chain,
  get_session_history,
  memory,
  input_messages_key="input",
  history_messages_key="history",
)

In [13]:
withMemoryChain.invoke(
  {"ability": "math", "input": "三角函数什么意思？?"},
  config={"configurable": {"session_id": "abc123"}},
)

------------------------------ enter-history ------------------------------
history messages:  [HumanMessage(content='What does cosine mean?'), AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a triangle.'), HumanMessage(content='What does cosine mean?'), AIMessage(content='Ratio of adjacent side to hypotenuse in a right triangle. Helps calculate angles and sides in trigonometry.')]
temp-memory:  []
------------------------------ exit-history ------------------------------


AIMessage(content='三角函数是一种描述角度和边长之间关系的数学函数，如正弦、余弦和正切。')

In [14]:
withMemoryChain.invoke(
  {"ability": "math", "input": "正弦是什么?"},
  config={"configurable": {"session_id": "abc123"}},
)

------------------------------ enter-history ------------------------------
history messages:  [HumanMessage(content='What does cosine mean?'), AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a triangle.'), HumanMessage(content='What does cosine mean?'), AIMessage(content='Ratio of adjacent side to hypotenuse in a right triangle. Helps calculate angles and sides in trigonometry.'), HumanMessage(content='三角函数什么意思？?'), AIMessage(content='三角函数是一种描述角度和边长之间关系的数学函数，如正弦、余弦和正切。')]
temp-memory:  [HumanMessage(content='三角函数什么意思？?'), AIMessage(content='三角函数是一种描述角度和边长之间关系的数学函数，如正弦、余弦和正切。')]
------------------------------ exit-history ------------------------------


AIMessage(content='正弦是一个三角函数，表示一个角的对边与斜边的比值。通常用sin表示，例如sin(30°) = 0.5。')

In [15]:
withMemoryChain.invoke(
  {"ability": "math", "input": "小学会学到吗?"},
  config={"configurable": {"session_id": "abc123"}},
)

------------------------------ enter-history ------------------------------
history messages:  [HumanMessage(content='What does cosine mean?'), AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a triangle.'), HumanMessage(content='What does cosine mean?'), AIMessage(content='Ratio of adjacent side to hypotenuse in a right triangle. Helps calculate angles and sides in trigonometry.'), HumanMessage(content='三角函数什么意思？?'), AIMessage(content='三角函数是一种描述角度和边长之间关系的数学函数，如正弦、余弦和正切。'), HumanMessage(content='正弦是什么?'), AIMessage(content='正弦是一个三角函数，表示一个角的对边与斜边的比值。通常用sin表示，例如sin(30°) = 0.5。')]
temp-memory:  [HumanMessage(content='三角函数什么意思？?'), AIMessage(content='三角函数是一种描述角度和边长之间关系的数学函数，如正弦、余弦和正切。'), HumanMessage(content='正弦是什么?'), AIMessage(content='正弦是一个三角函数，表示一个角的对边与斜边的比值。通常用sin表示，例如sin(30°) = 0.5。')]
------------------------------ exit-history ------------------------------


AIMessage(content='一般在初中阶段学习三角函数，小学阶段通常不包括正弦、余弦和正切等概念。')

In [21]:
store['abc123'].messages

[HumanMessage(content='What does cosine mean?'),
 AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a triangle.'),
 HumanMessage(content='What does cosine mean?'),
 AIMessage(content='Ratio of adjacent side to hypotenuse in a right triangle. Helps calculate angles and sides in trigonometry.'),
 HumanMessage(content='三角函数什么意思？?'),
 AIMessage(content='三角函数是一种描述角度和边长之间关系的数学函数，如正弦、余弦和正切。'),
 HumanMessage(content='正弦是什么?'),
 AIMessage(content='正弦是一个三角函数，表示一个角的对边与斜边的比值。通常用sin表示，例如sin(30°) = 0.5。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='一般在初中阶段学习三角函数，小学阶段通常不包括正弦、余弦和正切等概念。')]

In [18]:
memory.buffer_as_messages

[HumanMessage(content='正弦是什么?'),
 AIMessage(content='正弦是一个三角函数，表示一个角的对边与斜边的比值。通常用sin表示，例如sin(30°) = 0.5。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='一般在初中阶段学习三角函数，小学阶段通常不包括正弦、余弦和正切等概念。')]